In [1]:
import pandas as pd
import numpy as np
import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

In [12]:
df = pl.read_csv(
    "../data/processed/df_procesada.csv",
    separator=";",
    columns=["ESTANCIA", "COD_HOSPITAL", "DIAGNOSTICO1", "ANIO_EGRESO"],
)

In [25]:
resumen_egresos_nacionales = (
    df.group_by(pl.col(["ANIO_EGRESO", "COD_HOSPITAL", "DIAGNOSTICO1"]))
    .agg([pl.count().alias("n_egresos"), pl.col("ESTANCIA").sum().alias("dias_estada_totales")])
    .sort(by=["ANIO_EGRESO", "DIAGNOSTICO1", "n_egresos"], descending=True)
    .with_columns(
        pl.col("n_egresos")
        .rank(method="min", descending=True)
        .over(["ANIO_EGRESO", "DIAGNOSTICO1"])
        .alias("ranking_egresos")
    )
)

In [49]:
diags_ranking_1_o_2_torax = resumen_egresos_nacionales.filter(
    (pl.col("COD_HOSPITAL") == 112103)
    & ((pl.col("ranking_egresos")) == 1 | (pl.col("ranking_egresos") == 2))
).sort(["ANIO_EGRESO", "n_egresos"], descending=True)

In [53]:
diags_ranking_1_o_2_torax.group_by(pl.col(["ANIO_EGRESO"])).agg(
    pl.col("DIAGNOSTICO1").str.concat(", ")
)

ANIO_EGRESO,DIAGNOSTICO1
i64,str
2021,"""I35.0, Z45.0, …"
2020,"""Z51.4, Z29.1, …"
2019,"""I25.1, Z51.4, …"
